In [1]:
#Import Libraries
import tensorflow as tf
import numpy as np 
import cv2

C:\Users\Iremide\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Iremide\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
# capture object from the webcam (index 0)
cap = cv2.VideoCapture(0)

while cap.isOpened():
    # Capture frame-by-frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break  # Exit the loop if the frame was not captured successfully

    image = frame.copy()  # Create a copy of the captured frame
    height, width, _ = frame.shape  # Get the height and width of the frame

    # Convert the captured frame to a Tensor and resize it to 192x192 with padding
    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.image.resize_with_pad(input_image, 192, 192)

    # Load the pre-trained MoveNet model for pose estimation
    model_path = "movenet_lightning_fp16.tflite"
    interpreter = tf.lite.Interpreter(model_path)
    interpreter.allocate_tensors()  # Allocate memory for the model's tensors

    input_image = tf.cast(input_image, dtype=tf.uint8)  # Cast the image to uint8 type
    input_details = interpreter.get_input_details()  # Get model input details
    output_details = interpreter.get_output_details()  # Get model output details

    # Set the input tensor and run inference
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    interpreter.invoke()  # Perform inference
    keypoints = interpreter.get_tensor(output_details[0]['index'])  # Get the detected keypoints

    # Define the edges between keypoints for drawing
    KEYPOINT_EDGES = [(0, 1), (0, 2), (1, 3), (2, 4), (0, 5), (0, 6), (5, 7),
                      (7, 9), (6, 8), (8, 10), (5, 6), (5, 11), (6, 12), 
                      (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)]

    # Draw keypoints on the frame
    for keypoint in keypoints[0][0]:
        y, x, confidence = keypoint  # Extract y, x coordinates and confidence score
        if confidence > 0.3:  # Check if the confidence score is above the threshold
            cx = int(x * width)  # Convert normalized x-coordinate to pixel value
            cy = int(y * height)  # Convert normalized y-coordinate to pixel value
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # Draw keypoint as a red circle

    # Draw edges between keypoints
    for edge in KEYPOINT_EDGES:
        p1 = keypoints[0][0][edge[0]]  # Get the first keypoint in the edge
        p2 = keypoints[0][0][edge[1]]  # Get the second keypoint in the edge

        y1, x1, c1 = p1  # Extract y, x coordinates and confidence score of the first keypoint
        y2, x2, c2 = p2  # Extract y, x coordinates and confidence score of the second keypoint

        if c1 > 0.3 and c2 > 0.3:  # Only draw edges if confidence for both keypoints is high
            pt1 = (int(x1 * width), int(y1 * height))  # Convert first keypoint to pixel coordinates
            pt2 = (int(x2 * width), int(y2 * height))  # Convert second keypoint to pixel coordinates
            cv2.line(frame, pt1, pt2, (0, 255, 0), 2)  # Draw edge as a green line

    # Display the frame with drawn keypoints and edges
    cv2.imshow('MoveNet Lightning', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [5]:
from math import atan2, degrees

# Function to calculate angle between three points
def calculate_angle(p1, p2, p3):
    a = np.array(p1)
    b = np.array(p2)
    c = np.array(p3)
    
    ab = a - b
    bc = c - b
    
    angle = atan2(np.linalg.det([ab, bc]), np.dot(ab, bc))
    return abs(degrees(angle))

# Initialize variables for curl counting
left_curls_count = 0
right_curls_count = 0
left_stage = "down"
right_stage = "down"

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image = frame.copy()
    height, width, _ = frame.shape

    # Convert image to Tensor and resize with padding to the model's expected input size (192x192)
    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.image.resize_with_pad(input_image, 192, 192)

    model_path = "movenet_lightning_fp16.tflite"
    interpreter = tf.lite.Interpreter(model_path)
    interpreter.allocate_tensors()

    input_image = tf.cast(input_image, dtype=tf.uint8)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    interpreter.invoke()
    keypoints = interpreter.get_tensor(output_details[0]['index'])

    # Keypoint format: [y, x, score]
    KEYPOINT_EDGES = [(0, 1), (0, 2), (1, 3), (2, 4), (0, 5), (0, 6), (5, 7),
                      (7, 9), (6, 8), (8, 10), (5, 6), (5, 11), (6, 12), 
                      (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)]

    # Draw keypoints and edges back on the original frame
    for keypoint in keypoints[0][0]:
        y, x, confidence = keypoint
        if confidence > 0.3:  # Threshold for displaying keypoints
            cx = int(x * width)
            cy = int(y * height)
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)

    # Draw edges between keypoints
    for edge in KEYPOINT_EDGES:
        p1 = keypoints[0][0][edge[0]]
        p2 = keypoints[0][0][edge[1]]

        y1, x1, c1 = p1
        y2, x2, c2 = p2

        if c1 > 0.3 and c2 > 0.3:  # Only draw edges if confidence is high
            pt1 = (int(x1 * width), int(y1 * height))
            pt2 = (int(x2 * width), int(y2 * height))
            cv2.line(frame, pt1, pt2, (0, 255, 0), 2)

    # Extract keypoints for left and right arms
    left_shoulder = keypoints[0][0][5]
    left_elbow = keypoints[0][0][7]
    left_wrist = keypoints[0][0][9]

    right_shoulder = keypoints[0][0][6]
    right_elbow = keypoints[0][0][10]
    right_wrist = keypoints[0][0][5]

    # Calculate angles for left and right elbows
    left_elbow_angle = calculate_angle(
        (left_shoulder[1] * width, left_shoulder[0] * height),
        (left_elbow[1] * width, left_elbow[0] * height),
        (left_wrist[1] * width, left_wrist[0] * height)
    )
    
    right_elbow_angle = calculate_angle(
        (right_shoulder[1] * width, right_shoulder[0] * height),
        (right_elbow[1] * width, right_elbow[0] * height),
        (right_wrist[1] * width, right_wrist[0] * height)
    )

    # Left curl counting logic
    if left_elbow_angle > 160:
        left_stage = "down"
    if left_elbow_angle < 30 and left_stage == "down":
        left_curls_count += 1
        left_stage = "up"

    # Right curl counting logic
    #if right_elbow_angle > 160:
    #    right_stage = "down"
    #if right_elbow_angle < 30 and right_stage == "down":
    #    right_curls_count += 1
    #    right_stage = "up"

    # Show the frame with keypoints and edges
    cv2.putText(frame, f"Left Curls: {left_curls_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    #cv2.putText(frame, f"Right Curls: {right_curls_count}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('MoveNet Lightning', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()